In [1]:
# Importing packages

from astropy.io import fits # For handling fits files
from glob import glob # to search for file paths under similar names

import numpy as np
from matplotlib import pyplot as plt

# Importing all data

In [2]:
rootPath = r"/home/daraghhollman/jupyter/UCD_PASS_Labs/Astronomy_Image_Analysis"

## M91

In [3]:
# Note each group of files is a list for each band

# Bias
biasFiles_M91 = [glob(rootPath + r"/data_2019/M91/bias/*.fits")]

# Flats
flatsFiles_M91 = [
    glob(rootPath + r"/data_2019/M91/flats/B/*.fits"),
    glob(rootPath + r"/data_2019/M91/flats/V/*.fits"),
    glob(rootPath + r"/data_2019/M91/flats/Halpha_NII/*.fits")
]

# Object
objectFiles_M91 = [
    glob(rootPath + r"/data_2019/M91/object/B/*.fits"),
    glob(rootPath + r"/data_2019/M91/object/V/*.fits"),
    glob(rootPath + r"/data_2019/M91/object/Halpha_NII/*.fits")
]

In [4]:
# Load files using astropy.fits

biasFITS_M91 = [fits.open(el) for el in biasFiles_M91[0]]

flatFITS_M91 = [[fits.open(el) for el in flatsFiles_M91[0]], \
                [fits.open(el) for el in flatsFiles_M91[1]], \
                [fits.open(el) for el in flatsFiles_M91[2]]]

objectFITS_M91 = [[fits.open(el) for el in objectFiles_M91[0]], \
                [fits.open(el) for el in objectFiles_M91[1]], \
                [fits.open(el) for el in objectFiles_M91[2]]]

## NNSer

In [5]:
# Note, only frames of NNSer in the clear band.

# Bias
biasFiles_NNSer = [glob(rootPath + r"/data_2019/NNSer/bias/*.fits")]

# Flats
flatsFiles_NNSer = [glob(rootPath + r"/data_2019/NNSer/flats/clear/*.fits")]

# Object
objectFiles_NNSer = [glob(rootPath + r"/data_2019/NNSer/object/clear/*.fits")]

In [6]:
biasFITS_NNSer = [fits.open(el) for el in biasFiles_NNSer[0]]

flatFITS_NNSer = [fits.open(el) for el in flatsFiles_NNSer[0]]

objectFITS_NNSer = [fits.open(el) for el in objectFiles_NNSer[0]]

# Data Reduction

In [10]:
# Read in the pixel values of each bias into an np array. 
# Find the average value of each pixel across all bias frames. 
# Assign each average pixel to a new file: masterBias.fits


# Takes a list of arrays and returns an array with each element being
# the average element of the list
def createAvgArray(arraysList):
    
    arraySum = np.empty(np.shape(arraysList[0]))
    
    for array in arraysList:
        arraySum += array

    return arraySum / len(arraysList)

# Converts a list of fits files into a list of arrays and passes it to the createAvgArray
# function to get the masterBias
def createMasterBias(biasList):
    
    arraysList = list()
    
    for el in biasList:
        arraysList.append(el[0].data)
    
    avg = createAvgArray(arraysList)

    return avg

# Converts a list of fits files into a list of arrays, subtacting the masterBias,
# and passes it to the createAvgArray function to get the masterFlat
def createMasterFlat(flatsList, masterBias):
    
    arraysList = list()
    
    for el in flatsList:
        arraysList.append(el[0].data - masterBias)
    
    avgArray = createAvgArray(arraysList)
    
    # normalisation, find the value at the centre and divide by it
    xLength, yLength = np.shape(avgArray)
    
    normFac = avgArray[int(xLength/2)][int(yLength/2)]

    return avgArray / normFac

In [11]:
masterBias_M91 = createMasterBias(biasFITS_M91)
masterBias_NNSer = createMasterBias(biasFITS_NNSer)

In [12]:
masterFlats_M91_B = createMasterFlat(flatFITS_M91[0], masterBias_M91)
masterFlats_M91_V = createMasterFlat(flatFITS_M91[1], masterBias_M91)
masterFlats_M91_Ha = createMasterFlat(flatFITS_M91[2], masterBias_M91)

masterFlats_M91 = [masterFlats_M91_B, masterFlats_M91_V, masterFlats_M91_Ha]

masterFlat_NNSer = createMasterFlat(flatFITS_NNSer, masterBias_NNSer)